In [ ]:
from dummy_qua2 import *

In [ ]:
config = QMConfig(  # this config is illustrative and incomplete!!!
    channels=[
        IQOutputChannel(
            logical_address=LogicalVertex(0, 'q0', (0, 'xy')),
            port_i=('con1', 1),
            port_q=('con1', 2),
            waveforms=[
                IQWaveform(
                    name="sqrtX",
                    data=[0.3 + 3 * 1j, 0.4 - 2 * 1j]
                ),
                IQWaveform(
                    name="sqrtY",
                    data=[0.3 + 2 * 1j, 0.4 - 2 * 1j]
                ),
            ],
        ),
        SingleOutputChannel(
            logical_address=LogicalVertex(0, 'q0', (1, 'z')),
            port=('con1', 3),
            waveforms=[
                SingleWaveform(
                    name="z00",
                    data=[0.3, 0.5]
                ),
                SingleWaveform(
                    name="z01",
                    data=[0.6, 0.2]
                )
            ],
        ),
        IQOutputChannel(
            logical_address=LogicalVertex(0, 'q0', (2, 'm')),
            port_i=('con1', 1),
            port_q=('con1', 2),
            waveforms=[
                IQWaveform(
                    name="sqrtX",
                    data=[0.3 + 3 * 1j, 0.4 - 2 * 1j]
                ),
                IQWaveform(
                    name="sqrtY",
                    data=[0.3 + 2 * 1j, 0.4 - 2 * 1j]
                ),
            ],
        ),
        IQOutputChannel(
            logical_address=LogicalVertex(0, 'q0', (2, 'm')),
            port_i=('con1', 1),
            port_q=('con1', 2),
            waveforms=[
                IQWaveform(
                    name="meas",
                    data=[0.3 + 3 * 1j, 0.4 - 2 * 1j]
                ),
            ],
        ),
        IQInputChannel(
            logical_address=LogicalVertex(0, 'q1', (3, 'acquire')),
            port_i=('con1', 4),
            port_q=('con1', 5),
            integration_weights=[
                IQIntegrationWeight(
                    name="iw_i",
                    data=[0.3, 0.4]
                ),
                IQIntegrationWeight(
                    name="iw_q",
                    data=[0.3, 0.4 ]
                ),
            ],
        ),
        SingleOutputChannel(
            logical_address=LogicalVertex(0, 'q1', 'z'),
            name="z",
            port=('con1', 6),
            waveforms=[
                SingleWaveform(
                    name="z00",
                    data=[0.3, 0.5]
                ),
                SingleWaveform(
                    name="z01",
                    data=[0.6, 0.2]
                )
            ]
        ),
        SingleOutputChannel(
            logical_address=LogicalEdge(0, 'q1', 'q2', 'z'),
            port=('con1', 20),
            waveforms=[
                SingleWaveform(
                    name="coupling_z",
                    data=[0.3, 0.5]
                )
            ]
        )
    ]
)

# XEB example with 5 qubits (fig. 3 in google supremacy paper)

In [ ]:
# QUA program

@procedure
def reset_qubit(q):
    disc = declare(fixed)
    play('meas',
         logical_vertex(0, q, 'meas'),
         duration=100,
         oscillator=logical_vertex(0, q, 1),
         process=integration.full(target=disc,
                                  iw=0,
                                  input=logical_vertex(0, q, 'acquire'),
                                  oscillator=logical_vertex(0, q, 1))
         )

    with while_(disc > -0.005):
        play('x',
             logical_vertex(0, q, 'xy'),
             duration=22,
             oscillator=logical_vertex(0, q, 0))
        play('meas',
             logical_vertex(0, q, 'meas'),
             duration=100,
             oscillator=logical_vertex(0, q, 1),
             process=integration.full(target=disc,
                                      iw=0,
                                      input=logical_vertex(0, q, 'acquire'),
                                      oscillator=logical_vertex(0, q, 1)
                                      )
             )


@procedure
def two_qubit_uniatry_north(q1, q2):
    with parallel_():
        with thread_():
            play(waveform='z_north',
                 channel=logical_vertex(0, q1, 'z'),
                 duration=12)
        with thread_():
            play(waveform='coupler_on',
                 channel=logical_edge(0, q1, q2, 'z'),
                 duration=12)
        with thread_():
            play(waveform='z_south',
                 channel=logical_vertex(0, q2, 'z'),
                 duration=12)


@procedure
def two_qubit_uniatry_south(q1, q2):
    with parallel_():
        with thread_():
            play(waveform='z_south',
                 channel=logical_vertex(0, q1, 'z'),
                 duration=12)
        with thread_():
            play(waveform='coupler_on',
                 channel=logical_edge(0, q1, q2, 'z'),
                 duration=12)
        with thread_():
            play(waveform='z_north',
                 channel=logical_vertex(0, q2, 'z'),
                 duration=12)


@procedure
def two_qubit_uniatry_east(q1, q2):
    with parallel_():
        with thread_():
            play(waveform='z_east',
                 channel=logical_vertex(0, q1, 'z'),
                 duration=12)
        with thread_():
            play(waveform='coupler_on',
                 channel=logical_edge(0, q1, q2, 'z'),
                 duration=12)
        with thread_():
            play(waveform='z_west',
                 channel=logical_vertex(0, q2, 'z'),
                 duration=12)


@procedure
def two_qubit_uniatry_west(q1, q2):
    with parallel_():
        with thread_():
            play(waveform='z_west',
                 channel=logical_vertex(0, q1, 'z'),
                 duration=12)
        with thread_():
            play(waveform='coupler_on',
                 channel=logical_edge(0, q1, q2, 'z'),
                 duration=12)
        with thread_():
            play(waveform='z_east',
                 channel=logical_vertex(0, q2, 'z'),
                 duration=12)


@procedure
def random_single_all_qubits():
    with parallel_():
        with for_(element, range(5)):
            with thread_():
                assign(rand_single, random(3))
                play(waveform=rand_single,
                     channel=logical_vertex(0, element, 'xy'),
                     duration=25,
                     oscillator=logical_vertex(0, element, 0),
                     frame=logical_vertex(0, element, 0))


@procedure
def reset_all():
    with parallel_():
        with for_(element, range(5)):
            with thread_():
                if element == 3:
                    cond = True
                else:
                    cond = False
                reset_qubit(element, cond)


@procedure(return_type=QuaFloat(size=5))
def measure_all():
    q = declare(int)
    disc = declare(float, size=5)
    with for_(n, range(5)):
        play('meas',
             logical_vertex(0, q, 'meas'),
             duration=100,
             oscillator=logical_vertex(0, q, 1),
             process=integration.full(target=disc[q],
                                      iw=0,
                                      input=logical_vertex(0, q, 'acquire'),
                                      oscillator=logical_vertex(0, q, 1))
             )
    return disc


with program_():
    element = declare(int)
    n = declare(int)
    cycle = declare(int)
    rand_single = declare(int)
    with qrun_():
        with for_(n, range(1000)):
            reset_all()
            with for_(cycle, range(20)):  # this needs to be repeated for an increasing number of cycles
                random_single_all_qubits()
                two_qubit_uniatry_north(0, 1)
                random_single_all_qubits()
                two_qubit_uniatry_east(0, 2)
                random_single_all_qubits()
                two_qubit_uniatry_north(0, 3)
                random_single_all_qubits()
                two_qubit_uniatry_south(0, 4)
                random_single_all_qubits()
                two_qubit_uniatry_north(0, 1)
                random_single_all_qubits()
                two_qubit_uniatry_south(0, 4)
                random_single_all_qubits()
                two_qubit_uniatry_west(0, 1)
                random_single_all_qubits()
                two_qubit_uniatry_east(0, 2)
                random_single_all_qubits()
                two_qubit_uniatry_east(0, 2)
                random_single_all_qubits()
            measure_all()


        
                    
            
